In [2]:
!pip install -U ultralytics

In [3]:
import os
import shutil
import yaml
import glob
from ultralytics import YOLO

# ==========================================
# 1. SETUP: COPY DATA TO WRITABLE FOLDER
# ==========================================
INPUT_DIR = '/kaggle/input/ppeconstruction' 
WORKING_DIR = '/kaggle/working/ppe_data'

print("🔄 Copying dataset to writable directory...")

# Clean up previous runs
if os.path.exists(WORKING_DIR):
    shutil.rmtree(WORKING_DIR)

# Copy folder
shutil.copytree(INPUT_DIR, WORKING_DIR)
print(f"✅ Dataset copied to: {WORKING_DIR}")

# ==========================================
# 2. CLEAN LABELS (Remove 'No-Boots')
# ==========================================
print("🧹 Cleaning 'No-Boots'...")

# Find the YAML file
yaml_files = glob.glob(os.path.join(WORKING_DIR, '*.yaml'))
if not yaml_files:
    # Sometimes it's inside a subfolder, let's search recursively just in case
    yaml_files = glob.glob(os.path.join(WORKING_DIR, '**/*.yaml'), recursive=True)

if not yaml_files:
    raise FileNotFoundError("❌ No .yaml file found in dataset!")

yaml_file = yaml_files[0]
print(f"   Found config: {yaml_file}")

target_class_id = -1

with open(yaml_file, 'r') as f:
    data = yaml.safe_load(f)

# A. Find Class ID
raw_names = data['names']
names_dict = {i: n for i, n in enumerate(raw_names)} if isinstance(raw_names, list) else raw_names

for cid, cname in names_dict.items():
    if 'boot' in str(cname).lower() and 'no' in str(cname).lower():
        target_class_id = cid
        print(f"   🎯 Removing Class ID {cid}: {cname}")
        break

# B. Remove from Text Files
if target_class_id != -1:
    txt_files = glob.glob(f'{WORKING_DIR}/**/*.txt', recursive=True)
    for txt in txt_files:
        with open(txt, 'r') as f: lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.split()
            if not parts: continue
            cls = int(parts[0])
            if cls == target_class_id: continue 
            if cls > target_class_id: parts[0] = str(cls - 1) 
            new_lines.append(" ".join(parts) + "\n")
        with open(txt, 'w') as f: f.writelines(new_lines)

    # Update Names
    new_names = {i: n for i, n in enumerate([v for k,v in names_dict.items() if k != target_class_id])}
    data['names'] = new_names
    data['nc'] = len(new_names)

# ==========================================
# 3. FIX PATHS (Crucial for your structure)
# ==========================================
# Your structure is: /kaggle/working/ppe_data/images/train
# So 'path' = /kaggle/working/ppe_data
# And 'train' = images/train

data['path'] = WORKING_DIR  # Absolute Root
data['train'] = 'images/train'
data['val'] = 'images/val'   # Usually 'valid' or 'val', checking...
data['test'] = 'images/test'

# Verify if 'val' folder is named 'valid' instead
if os.path.exists(os.path.join(WORKING_DIR, 'images/valid')):
    data['val'] = 'images/valid'

if 'download' in data: del data['download']

# Save Fixed YAML
with open(yaml_file, 'w') as f:
    yaml.dump(data, f)

print(f"✅ YAML Paths Fixed: train={data['train']}, val={data['val']}")

# ==========================================
# 4. TRAIN (2x GPU)
# ==========================================
# print("🔥 Starting YOLOv11m Training...")

# model = YOLO('yolo11m.pt')

# results = model.train(
#     data=yaml_file,
#     # --- The "Winning Recipe" ---
#     epochs=300,                # Paper used long training (stops early if needed)
#     patience=50,               # Stop if no improvement for 50 epochs
#     batch=32,                  # High batch size for stable gradients
#     imgsz=640,
    
#     # Optimizer Settings (Crucial for 95% mAP)
#     optimizer='SGD',           # Research standard (often beats Adam for YOLO)
#     lr0=0.01,                  # Initial Learning Rate
#     lrf=0.01,                  # Final Learning Rate (Cosine decay)
#     momentum=0.937,            # High momentum
#     weight_decay=0.0005,       # Regularization to prevent overfitting
    
#     # Augmentation (To fix your class imbalance)
#     augment=True,
#     mosaic=1.0,                # 100% Mosaic (Essential for small objects like goggles)
#     mixup=0.15,                # 15% Mixup (Fixes the "No-Helmet" imbalance)
#     copy_paste=0.1,            # 10% Copy-Paste (Standard in 2024 papers)
    
#     # Hardware
#     device=[0, 1],             # Use both Kaggle GPUs
#     project='/kaggle/working/runs',
#     name='paper_replication_run'
# )

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🔄 Copying dataset to writable directory...
✅ Dataset copied to: /kaggle/working/ppe_data
🧹 Cleaning 'No-Boots'...
   Found config: /kaggle/working/ppe_data/data.yaml
   🎯 Removing Class ID 10: no_boots
✅ YAML Paths Fixed: train=images/train, val=images/val


In [ ]:
!zip -r sgd_trained_yolo11m.zip /kaggle/working/runs

In [ ]:
!pip install ipython

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink('sgd_trained_yolo11m.zip')

In [7]:
def train_with_optimizer(optimizer_type):
    """
    Train YOLOv11m with different optimizers
    """
    from ultralytics import YOLO
    
    model = YOLO('yolo11m.pt')
    
    # Training configurations
    if optimizer_type == 'adamw':
        results = model.train(
            data=yaml_file,
            epochs=150,
            batch=32,
            optimizer='AdamW',
            lr0=0.001,
            lrf=0.01,
            name=f'exp_adamw',
            device=[0,1],
            project='/kaggle/working/runs_adamw'
        )
    elif optimizer_type == 'sgd':
        results = model.train(
            data=yaml_file,
            epochs=150,
            batch=32,
            optimizer='SGD',
            lr0=0.01,
            momentum=0.937,
            weight_decay=0.0005,
            name=f'exp_sgd',
            device=[0, 1],             # Use both Kaggle GPUs
            project='/kaggle/working/runs_sgd'

        )
    
    return results


In [8]:
results_adamw = train_with_optimizer('adamw')
results_sgd = train_with_optimizer('sgd')

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/ppe_data/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp_adamw, nbs=64, nms=False, opset=None, optimi

In [11]:
import pandas as pd
# Compare results
comparison = pd.DataFrame([
    {
        'Optimizer': 'AdamW',
        'mAP@50': results_adamw.results_dict['metrics/mAP50(B)'],
        'No-Helmet Recall': results_adamw.results_dict['metrics/recall(No-Helmet)'],
        'Training Time (h)': results_adamw.training_time / 3600
    },
    {
        'Optimizer': 'SGD',
        'mAP@50': results_sgd.results_dict['metrics/mAP50(B)'],
        'No-Helmet Recall': results_sgd.results_dict['metrics/recall(No-Helmet)'],
        'Training Time (h)': results_sgd.training_time / 3600
    }
])

print(comparison.to_markdown(index=False))

AttributeError: 'NoneType' object has no attribute 'results_dict'

In [16]:
!zip -r runs_sgd.zip /kaggle/working/runs_adamw

  adding: kaggle/working/runs_adamw/ (stored 0%)
  adding: kaggle/working/runs_adamw/exp_adamw/ (stored 0%)
  adding: kaggle/working/runs_adamw/exp_adamw/labels.jpg (deflated 28%)
  adding: kaggle/working/runs_adamw/exp_adamw/weights/ (stored 0%)
  adding: kaggle/working/runs_adamw/exp_adamw/weights/last.pt (deflated 8%)
  adding: kaggle/working/runs_adamw/exp_adamw/weights/best.pt (deflated 8%)
  adding: kaggle/working/runs_adamw/exp_adamw/results.png (deflated 8%)
  adding: kaggle/working/runs_adamw/exp_adamw/results.csv (deflated 61%)
  adding: kaggle/working/runs_adamw/exp_adamw/confusion_matrix.png (deflated 19%)
  adding: kaggle/working/runs_adamw/exp_adamw/train_batch1.jpg (deflated 2%)
  adding: kaggle/working/runs_adamw/exp_adamw/confusion_matrix_normalized.png (deflated 16%)
  adding: kaggle/working/runs_adamw/exp_adamw/val_batch1_pred.jpg (deflated 12%)
  adding: kaggle/working/runs_adamw/exp_adamw/train_batch5041.jpg (deflated 9%)
  adding: kaggle/working/runs_adamw/exp_ada

In [13]:
!pip install ipython

In [17]:
from IPython.display import FileLink

In [18]:
FileLink('runs_sgd.zip')

/kaggle/working/runs_sgd.zip

In [19]:
from ultralytics import YOLO
import pandas as pd

# Load the already-trained models from your previous training runs
try:
    model_adamw = YOLO('/kaggle/working/runs_adamw/exp_adamw/weights/best.pt')
    model_sgd = YOLO('/kaggle/working/runs_sgd/exp_sgd/weights/best.pt')
    
    # Run validation to get metrics (this is fast, not retraining)
    print("Evaluating AdamW model...")
    metrics_adamw = model_adamw.val()  # This evaluates on validation set
    
    print("Evaluating SGD model...")
    metrics_sgd = model_sgd.val()  # This evaluates on validation set
    
    # Get training times from your previous runs (you'll need to note these down)
    # If you didn't record them, estimate or leave blank
    time_adamw = 3 * 3600  # Estimate: 3 hours in seconds
    time_sgd = 3 * 3600    # Estimate: 3 hours in seconds
    
    # Create comparison DataFrame - check the actual keys in metrics
    print("\nAvailable keys in metrics_adamw:", dir(metrics_adamw))
    
    # Try different key patterns based on Ultralytics version
    comparison = pd.DataFrame([
        {
            'Optimizer': 'AdamW',
            'mAP@50': getattr(metrics_adamw, 'box', {}).get('map50', None) or 
                      getattr(metrics_adamw, 'results_dict', {}).get('metrics/mAP50(B)', None) or
                      getattr(metrics_adamw, 'maps', [None])[0],
            'mAP@50-95': getattr(metrics_adamw, 'box', {}).get('map', None) or 
                        getattr(metrics_adamw, 'results_dict', {}).get('metrics/mAP50-95(B)', None),
            'Precision': getattr(metrics_adamw, 'box', {}).get('precision', None) or 
                        getattr(metrics_adamw, 'results_dict', {}).get('metrics/precision', None),
            'Recall': getattr(metrics_adamw, 'box', {}).get('recall', None) or 
                     getattr(metrics_adamw, 'results_dict', {}).get('metrics/recall', None),
            'Training Time (h)': time_adamw / 3600
        },
        {
            'Optimizer': 'SGD',
            'mAP@50': getattr(metrics_sgd, 'box', {}).get('map50', None) or 
                     getattr(metrics_sgd, 'results_dict', {}).get('metrics/mAP50(B)', None) or
                     getattr(metrics_sgd, 'maps', [None])[0],
            'mAP@50-95': getattr(metrics_sgd, 'box', {}).get('map', None) or 
                        getattr(metrics_sgd, 'results_dict', {}).get('metrics/mAP50-95(B)', None),
            'Precision': getattr(metrics_sgd, 'box', {}).get('precision', None) or 
                        getattr(metrics_sgd, 'results_dict', {}).get('metrics/precision', None),
            'Recall': getattr(metrics_sgd, 'box', {}).get('recall', None) or 
                     getattr(metrics_sgd, 'results_dict', {}).get('metrics/recall', None),
            'Training Time (h)': time_sgd / 3600
        }
    ])
    
    print("\n" + comparison.to_markdown(index=False))
    
except Exception as e:
    print(f"Error loading models: {e}")
    
    # Alternative: Read from CSV files if they exist
    print("\nTrying to read from CSV files...")
    import os
    
    def read_metrics_from_csv(optimizer_type):
        csv_path = f'/kaggle/working/runs_{optimizer_type}/exp_{optimizer_type}/results.csv'
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            last_row = df.iloc[-1]
            return {
                'mAP@50': last_row.get('metrics/mAP50(B)', last_row.get('metrics/mAP50', None)),
                'mAP@50-95': last_row.get('metrics/mAP50-95(B)', last_row.get('metrics/mAP50-95', None)),
                'Precision': last_row.get('metrics/precision', None),
                'Recall': last_row.get('metrics/recall', None)
            }
        return None
    
    adamw_metrics = read_metrics_from_csv('adamw')
    sgd_metrics = read_metrics_from_csv('sgd')
    
    if adamw_metrics and sgd_metrics:
        comparison = pd.DataFrame([
            {'Optimizer': 'AdamW', **adamw_metrics, 'Training Time (h)': 3},
            {'Optimizer': 'SGD', **sgd_metrics, 'Training Time (h)': 3}
        ])
        print(comparison.to_markdown(index=False))

Evaluating AdamW model...
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,037,742 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1909.9±562.5 MB/s, size: 85.8 KB)
val: Scanning /kaggle/working/ppe_data/labels/val.cache... 143 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 143/143 323.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 1.8it/s 4.9s0.5ss
                   all        143       1168      0.652      0.621      0.632      0.317
                helmet        107        201      0.851      0.816      0.855      0.463
                gloves         68        136      0.817      0.722      0.782      0.364
                  vest        109        171      0.842      0.801      0.851      0.528
                 boots         64        151      0.832      0.755      0.809      0.462
        

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os

def augmentation_ablation_study():
    """
    Test different augmentation strategies
    """
    # Create figures directory if it doesn't exist
    os.makedirs('figures', exist_ok=True)
    
    configs = [
        {'mosaic': 0.0, 'mixup': 0.0, 'name': 'baseline'},
        {'mosaic': 1.0, 'mixup': 0.0, 'name': 'mosaic_only'},
        {'mosaic': 0.0, 'mixup': 0.15, 'name': 'mixup_only'},
        {'mosaic': 1.0, 'mixup': 0.15, 'name': 'combined'},
    ]
    
    results = []
    
    for cfg in configs:
        print(f"\n{'='*50}")
        print(f"Training with configuration: {cfg['name']}")
        print(f"mosaic={cfg['mosaic']}, mixup={cfg['mixup']}")
        print('='*50)
        
        model = YOLO('yolo11m.pt')
        
        # Train the model
        training_results = model.train(
            data='ppe_dataset.yaml',
            epochs=50,  # Shorter for ablation
            imgsz=640,
            mosaic=cfg['mosaic'],
            mixup=cfg['mixup'],
            name=f"aug_{cfg['name']}",
            project='augmentation_ablation',
            save=True,
            exist_ok=True
        )
        
        # Load the trained model for validation
        model_path = f'augmentation_ablation/aug_{cfg["name"]}/weights/best.pt'
        trained_model = YOLO(model_path)
        
        # Validate to get metrics
        val_results = trained_model.val()
        
        # Extract metrics - check what's available
        print(f"\nAvailable attributes in val_results for {cfg['name']}:")
        print(dir(val_results))
        
        # Different ways to access metrics based on Ultralytics version
        if hasattr(val_results, 'box'):
            # For newer versions
            overall_map = val_results.box.map50  # mAP@50
            no_helmet_map = None  # You'd need class-specific metrics
        elif hasattr(val_results, 'results_dict'):
            # For older versions
            overall_map = val_results.results_dict.get('metrics/mAP50(B)', None)
            no_helmet_map = val_results.results_dict.get('metrics/mAP50(No-Helmet)', None)
        else:
            # Try to get from CSV
            csv_path = f'augmentation_ablation/aug_{cfg["name"]}/results.csv'
            if os.path.exists(csv_path):
                df = pd.read_csv(csv_path)
                last_row = df.iloc[-1]
                overall_map = last_row.get('metrics/mAP50(B)', None)
                no_helmet_map = last_row.get('metrics/mAP50(No-Helmet)', None)
            else:
                overall_map = None
                no_helmet_map = None
        
        results.append({
            'Configuration': cfg['name'],
            'Mosaic': cfg['mosaic'],
            'Mixup': cfg['mixup'],
            'Overall mAP@50': overall_map,
            'No-Helmet mAP@50': no_helmet_map,
            'Model Path': model_path
        })
        
        print(f"Results for {cfg['name']}:")
        print(f"  Overall mAP@50: {overall_map}")
        print(f"  No-Helmet mAP@50: {no_helmet_map}")
    
    # Create results DataFrame
    df = pd.DataFrame(results)
    print("\n" + df.to_markdown(index=False))
    
    # Save results to CSV
    df.to_csv('augmentation_ablation_results.csv', index=False)
    
    # Visualize - only if we have valid data
    if len(df) > 0 and df['Overall mAP@50'].notna().any():
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        # Plot 1: Overall mAP
        axes[0].bar(df['Configuration'], df['Overall mAP@50'])
        axes[0].set_title('Impact of Augmentation on Overall mAP@50')
        axes[0].set_ylabel('mAP@50')
        axes[0].set_xlabel('Configuration')
        axes[0].tick_params(axis='x', rotation=45)
        
        # Add value labels on bars
        for i, v in enumerate(df['Overall mAP@50']):
            if pd.notna(v):
                axes[0].text(i, v, f'{v:.3f}', ha='center', va='bottom')
        
        # Plot 2: No-Helmet mAP (if available)
        if df['No-Helmet mAP@50'].notna().any():
            axes[1].bar(df['Configuration'], df['No-Helmet mAP@50'])
            axes[1].set_title('Impact of Augmentation on No-Helmet mAP@50')
            axes[1].set_ylabel('mAP@50')
            axes[1].set_xlabel('Configuration')
            axes[1].tick_params(axis='x', rotation=45)
            
            # Add value labels on bars
            for i, v in enumerate(df['No-Helmet mAP@50']):
                if pd.notna(v):
                    axes[1].text(i, v, f'{v:.3f}', ha='center', va='bottom')
        else:
            axes[1].text(0.5, 0.5, 'No-Helmet metrics not available\nCheck class-specific metrics in results',
                        ha='center', va='center', transform=axes[1].transAxes)
            axes[1].set_title('No-Helmet mAP@50 (Not Available)')
        
        plt.tight_layout()
        plt.savefig('figures/augmentation_ablation.png', dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print("\nWarning: No valid mAP data to visualize!")
    
    return df

# Alternative: If you just want to analyze existing runs without retraining
def analyze_existing_augmentation_runs():
    """Analyze previously trained augmentation runs without retraining"""
    
    configs = ['baseline', 'mosaic_only', 'mixup_only', 'combined']
    results = []
    
    for config_name in configs:
        run_dir = f'augmentation_ablation/aug_{config_name}'
        
        if os.path.exists(run_dir):
            print(f"\nAnalyzing {config_name}...")
            
            # Try to load the model
            model_path = f'{run_dir}/weights/best.pt'
            if os.path.exists(model_path):
                try:
                    model = YOLO(model_path)
                    val_results = model.val()
                    
                    # Extract metrics
                    if hasattr(val_results, 'box'):
                        overall_map = val_results.box.map50
                        # Try to get class-specific metrics
                        if hasattr(val_results, 'ap_class_index'):
                            print(f"Class indices: {val_results.ap_class_index}")
                    elif hasattr(val_results, 'results_dict'):
                        overall_map = val_results.results_dict.get('metrics/mAP50(B)', None)
                    else:
                        overall_map = None
                except Exception as e:
                    print(f"Error validating model: {e}")
                    overall_map = None
            else:
                overall_map = None
                print(f"Model not found at {model_path}")
            
            # Try to read from CSV
            csv_path = f'{run_dir}/results.csv'
            if os.path.exists(csv_path):
                df_csv = pd.read_csv(csv_path)
                last_row = df_csv.iloc[-1]
                csv_map = last_row.get('metrics/mAP50(B)', None)
                
                # Use CSV data if model validation failed
                if overall_map is None:
                    overall_map = csv_map
            else:
                csv_map = None
            
            results.append({
                'Configuration': config_name,
                'Overall mAP@50': overall_map
            })
        else:
            print(f"Run directory not found: {run_dir}")
            results.append({
                'Configuration': config_name,
                'Overall mAP@50': None
            })
    
    df = pd.DataFrame(results)
    print("\n" + df.to_markdown(index=False))
    return df

# Run the study
if __name__ == "__main__":
    # Option 1: Run full ablation study (will train 4 models)
    # results_df = augmentation_ablation_study()
    
    # Option 2: Just analyze existing runs (no training)
    results_df = analyze_existing_augmentation_runs()

Run directory not found: augmentation_ablation/aug_baseline
Run directory not found: augmentation_ablation/aug_mosaic_only
Run directory not found: augmentation_ablation/aug_mixup_only
Run directory not found: augmentation_ablation/aug_combined

| Configuration   | Overall mAP@50   |
|:----------------|:-----------------|
| baseline        |                  |
| mosaic_only     |                  |
| mixup_only      |                  |
| combined        |                  |
